In [0]:
%run ../configurations/paths_config

In [0]:
%run ../utils/etl_support_functions

In [0]:
dbutils.widgets.text("File Date","2021-03-21")
v_file_date = dbutils.widgets.get("File Date")

In [0]:
from pyspark.sql.types import StringType,IntegerType,DoubleType,TimestampType,DateType,StructType,StructField,DoubleType
from pyspark.sql.functions import col,lit
from pyspark.sql import functions as sf
import json
from delta.tables import DeltaTable


In [0]:
new_schema = StructType(fields=[StructField("driverId",IntegerType(),False),
                                StructField("lap",IntegerType(),True),
                                StructField("milliseconds",IntegerType(),True),
                                StructField("raceId",IntegerType(),True),
                                StructField("stop",IntegerType(),True),
                                StructField("time",StringType(),True),
                                ])

In [0]:
raw_df = spark.read.schema(new_schema).option("multiline",True).json(raw_path+f"/{v_file_date}/pit_stops.json") 

In [0]:
renamed_df = raw_df.withColumnRenamed("driverId","driver_id").withColumnRenamed("raceId","race_id")

In [0]:
audited_df = renamed_df.withColumn("ingestion_timestamp",sf.current_timestamp())

In [0]:
upsert_into_delta_table(audited_df, 'vsarthicat.formula1_silver.pit_stops','race_id')

In [0]:
%sql
SELECT race_id, COUNT(*) 
FROM formula1_silver.pit_stops 
GROUP BY race_id
ORDER BY race_id DESC
LIMIT(20);
